In [2]:
import requests
import os
from google.auth import default
from google.auth.transport.requests import Request
from google.cloud import optimization_v1
import googlemaps
from dotenv import load_dotenv

load_dotenv()
proxy_address = "http://127.0.0.1:7890"
# Set the environment variables to specify the proxy
os.environ["HTTP_PROXY"] = proxy_address
os.environ["HTTPS_PROXY"] = proxy_address
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/kaicheng/ProjectsFormal/wheelTrans/secrets/key.json"
os.environ["PROJECT_ID"] = "elderlyhometransportation"
googleMapKey = os.getenv("GOOGLE_MAP_API_KEY")

ROOT_PATH = os.path.dirname(os.path.abspath(''))
ROOT_PATH


'/Users/kaicheng/ProjectsFormal/wheelTrans/src'

In [3]:
def call_sync_api() -> None:
    request_file_name = "resources/request.json"
    fleet_routing_client = optimization_v1.FleetRoutingClient()

    with open(request_file_name) as f:
        fleet_routing_request = optimization_v1.OptimizeToursRequest.from_json(f.read())
        fleet_routing_request.parent = f"projects/elderlyhometransportation"
        fleet_routing_response = fleet_routing_client.optimize_tours(
            fleet_routing_request, timeout=10
        )
        return fleet_routing_response


def get_direction(origin, destination, waypoints):
    waypoints_str = "|".join(["optimize:true"] + waypoints)
    base_url = "https://maps.googleapis.com/maps/api/directions/json"
    google_map_api_key = os.getenv("GOOGLE_MAP_API_KEY")

    params = {
        "destination": destination,
        "origin": origin,
        "waypoints": waypoints_str,
        "key": google_map_api_key
    }

    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        return response.text
    else:
        print(f"Request failed with status code: {response.status_code}")
        return None


def get_geocoding(gmaps, address):
    result = gmaps.geocode(address)
    if not result or len(result) != 1:
        print(f"Failed to get geocoding for {address}")
        return None
    location = result[0]['geometry']['location']
    lat = location["lat"]
    lng = location["lng"]
    return [lat, lng]


origin = "8408 Garvey Ave. #101 Rosemead, CA 91770"
destination = "8408 Garvey Ave. #101 Rosemead, CA 91770"
address1 = "3843 Maxson Road #226 El Monte, CA 91732"
address2 = "119 Garcelon Ave Apt B Monterey Park, CA 91754"
waypoints = [origin, address1, address2, destination]
# get_direction(origin, destination, waypoints, destination)
# get_geocoding(origin)
# get_geocoding(address1)
# get_geocoding(address2)
# gmaps = googlemaps.Client(key=os.getenv("GOOGLE_MAP_API_KEY"))
# coordinates = []
# for location in waypoints:
#     coordinates.append(get_geocoding(gmaps, location))
# print(coordinates)
# response = call_sync_api()


In [9]:
import pandas

df = pandas.read_excel("resources/TransportList.xlsx")
addresses = addresses = df['Address'].dropna().tolist()
#dev short
addresses = addresses[:10]
addresses

['3843 Maxson Road #226  El Monte, CA 91732',
 '119 Garcelon Ave Apt B  Monterey Park, CA 91754',
 '119 Garcelon Ave Apt B  Monterey Park, CA 91754',
 '3843 Maxson Rd Apt 314  El Monte, CA 91732',
 '3843 Maxson Rd Apt 314  El Monte, CA 91732',
 '2417 Angelus Ave #306 6266880563 Rosemead, CA 91770',
 '201 1/2 W Newmark Ave  Monterey Park, CA 91755',
 '209 W. Camino Real Ave  Arcadia, CA 91007',
 '209 W. Camino Real Ave  Arcadia, CA 91007',
 '3436 Tyler Ave #409  El Monte, CA 91731']

In [24]:
gmaps = googlemaps.Client(key=os.getenv("GOOGLE_MAP_API_KEY"))
coordinates = []
validAddresses = []
invalidAddresses = []
for address in addresses:
    latlng = get_geocoding(gmaps, address)
    validAddresses.append(address)
    if latlng:
        coordinates.append(latlng)
        invalidAddresses.append(address)
    else:
        print(f"Failed to get geocoding for {address}")
coordinates

[[34.072393, -118.0102374],
 [34.0651666, -118.12373],
 [34.0651666, -118.12373],
 [34.072393, -118.0102374],
 [34.072393, -118.0102374],
 [34.0564491, -118.0874686],
 [34.0600187, -118.1244518],
 [34.122069, -118.0391543],
 [34.122069, -118.0391543],
 [34.0709251, -118.0357052]]

In [11]:
from resources.vehicles import vehicles
import json  

shipments = list(map(lambda latlng:
                     {
                         "loadDemands": {
                             "weight": {
                                 "amount": "1"
                             }
                         },
                         "pickups": [
                             {
                                 "arrivalLocation": {
                                     "latitude": latlng[0],
                                     "longitude": latlng[1]
                                 },
                                 "duration": "150s",
                                #  "timeWindows": [
                                #      {
                                #          "endTime": "1970-01-01T00:33:20Z",
                                #          "startTime": "1970-01-01T00:16:40Z"
                                #      }
                                #  ]
                             }
                         ]
                     }
                     , coordinates))
requestDict = {
    "parent": "projects/elderlyhometransportation",
    "model": {
        "shipments": shipments,
        "vehicles": vehicles
    }
}
requestJson = json.dumps(requestDict)

requestJson

'{"parent": "projects/elderlyhometransportation", "model": {"shipments": [{"loadDemands": {"weight": {"amount": "1"}}, "pickups": [{"arrivalLocation": {"latitude": 34.072393, "longitude": -118.0102374}, "duration": "150s"}]}, {"loadDemands": {"weight": {"amount": "1"}}, "pickups": [{"arrivalLocation": {"latitude": 34.0651666, "longitude": -118.12373}, "duration": "150s"}]}, {"loadDemands": {"weight": {"amount": "1"}}, "pickups": [{"arrivalLocation": {"latitude": 34.0651666, "longitude": -118.12373}, "duration": "150s"}]}, {"loadDemands": {"weight": {"amount": "1"}}, "pickups": [{"arrivalLocation": {"latitude": 34.072393, "longitude": -118.0102374}, "duration": "150s"}]}, {"loadDemands": {"weight": {"amount": "1"}}, "pickups": [{"arrivalLocation": {"latitude": 34.072393, "longitude": -118.0102374}, "duration": "150s"}]}, {"loadDemands": {"weight": {"amount": "1"}}, "pickups": [{"arrivalLocation": {"latitude": 34.0564491, "longitude": -118.0874686}, "duration": "150s"}]}, {"loadDemands":

In [14]:
fleet_routing_client = optimization_v1.FleetRoutingClient()
fleetOptimizationRequest = optimization_v1.OptimizeToursRequest.from_json(requestJson)

response = fleet_routing_client.optimize_tours(
    fleetOptimizationRequest, timeout=10
)
response

routes {
  vehicle_start_time {
  }
  vehicle_end_time {
    seconds: 4643
  }
  visits {
    shipment_index: 4
    is_pickup: true
    start_time {
      seconds: 794
    }
    load_demands {
      key: "weight"
      value {
        amount: 1
      }
    }
    detour {
    }
    arrival_loads {
      type_: "weight"
    }
    demands {
      type_: "weight"
      value: 1
    }
  }
  visits {
    shipment_index: 3
    is_pickup: true
    start_time {
      seconds: 944
    }
    load_demands {
      key: "weight"
      value {
        amount: 1
      }
    }
    detour {
      seconds: 150
    }
    arrival_loads {
      type_: "weight"
      value: 1
    }
    demands {
      type_: "weight"
      value: 1
    }
  }
  visits {
    shipment_index: 2
    is_pickup: true
    start_time {
      seconds: 1938
    }
    load_demands {
      key: "weight"
      value {
        amount: 1
      }
    }
    detour {
      seconds: 1367
    }
    arrival_loads {
      type_: "weight"
      val

In [27]:
for vehicleRoutes in response.routes:
    shipmentIds = list(map(lambda visit: visit.shipment_index, vehicleRoutes.visits))
    addressOrders = list(map(lambda index: validAddresses[index], shipmentIds))
    print({
        'vehicle': vehicleRoutes.vehicle_index or 0,
        'order':addressOrders
        })


{'vehicle': 0, 'order': ['3843 Maxson Rd Apt 314  El Monte, CA 91732', '3843 Maxson Rd Apt 314  El Monte, CA 91732', '119 Garcelon Ave Apt B  Monterey Park, CA 91754', '3843 Maxson Road #226  El Monte, CA 91732', '119 Garcelon Ave Apt B  Monterey Park, CA 91754']}
{'vehicle': 1, 'order': ['3436 Tyler Ave #409  El Monte, CA 91731', '209 W. Camino Real Ave  Arcadia, CA 91007', '209 W. Camino Real Ave  Arcadia, CA 91007', '2417 Angelus Ave #306 6266880563 Rosemead, CA 91770', '201 1/2 W Newmark Ave  Monterey Park, CA 91755']}


In [10]:
routes = response.routes
routesPb0 = response.routes[0]
routesPb1 = response.routes[1]
a = list(map(lambda x: {"vehicleIndex": x.vehicle_index}, routes))
